# Modèle pour la détermination du type de véhicule

### 1. Préparation des données

Transformation : Niveau de gris sur tout le dataset

In [ ]:
import cv2, os

output_folder = './gray_images'

for image_name in os.listdir('./result') :
    img = cv2.imread(f'./result/{image_name}')
    grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, grayFrame)


Transformation : Floutage des images de meilleure qualité

Transformation : UpSampling en retournant les images aléatoirement

In [13]:
import tensorflow as tf
import numpy as np

def random_rotate_image(opencv_img):
    img = tf.convert_to_tensor(opencv_img, dtype=tf.float32)
    img = tf.image.rot90(img)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)

    img_rotated = img.numpy().astype(np.uint8)
    return img_rotated

output_folder = './rotated_gray_images'
os.makedirs(output_folder, exist_ok=True)

for image_name in os.listdir('./gray_images') :
    img = cv2.imread(f'./gray_images/{image_name}')

    rotations_list = [random_rotate_image(img), random_rotate_image(img)]
    for i, rotated_image in enumerate(rotations_list) :
        output_image_name = f"{os.path.splitext(image_name)[0]}_{i}.png"
        output_path = os.path.join(output_folder, output_image_name)
        cv2.imwrite(output_path, rotated_image)

Importation des datasets

In [10]:
import os
import tensorflow as tf

# Définition des chemins pour les fichiers image
train_dir = "./rotated_gray_images"
val_dir = ""
test_dir = ""

# Définition des cosntantes 
batch_size = 32
img_height = 224  # Hauteur de l'image (modifiable selon vos besoins)
img_width = 224 

x_train = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode=None  # Pas de labels automatiques
    )

file_paths = x_train.file_paths

# Extraire les labels depuis les noms de fichiers
image_paths = [os.path.basename(path) for path in file_paths]
y_train = [path.split('_')[0] for path in image_paths] 

print(y_train[:10])

Found 1528 files belonging to 1 classes.
['cars', 'cars', 'truck', 'cars', 'cars', 'cars', 'cars', 'truck', 'cars', 'truck']


Création du modèle